## Deliverable 2. Create a Customer Travel Destinations Map.

In [10]:
# Import the dependencies.
import pandas as pd
import gmaps
import ipywidgets as widgets
import requests
import gmaps.datasets

# Import the API key.
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [11]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Atuona,PF,-9.8000,-139.0333,79.75,75,94,15.37,light rain
1,1,Turbat,PK,26.0023,63.0440,99.68,18,67,9.42,broken clouds
2,2,Inta,RU,66.0317,60.1659,31.12,88,98,6.85,overcast clouds
3,3,Hobart,AU,-42.8794,147.3294,57.20,67,75,5.75,light rain
4,4,Saint George,US,37.1041,-113.5841,53.60,50,1,10.36,clear sky


In [12]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 85


In [13]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Atuona,PF,-9.8000,-139.0333,79.75,75,94,15.37,light rain
5,5,Vaini,TO,-21.2000,-175.2000,73.40,94,75,9.22,broken clouds
9,9,Rikitea,PF,-23.1203,-134.9692,76.33,73,71,9.95,light rain
14,14,Lastoursville,GA,-0.8143,12.7287,73.40,94,75,1.45,fog
15,15,Butaritari,KI,3.0707,172.7902,82.35,77,16,9.01,moderate rain
16,16,Hilo,US,19.7297,-155.0900,77.00,69,90,4.61,overcast clouds
23,23,Busselton,AU,-33.6500,115.3333,77.00,57,11,1.01,few clouds
25,25,Opuwo,NaN,-18.0607,13.8400,77.32,32,0,4.97,clear sky
31,31,Kingsville,US,27.5159,-97.8561,78.80,83,90,16.11,overcast clouds
33,33,Poum,NC,-20.2333,164.0167,76.15,57,36,20.65,scattered clouds


In [15]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                208
City                   208
Country                206
Lat                    208
Lng                    208
Max Temp               208
Humidity               208
Cloudiness             208
Wind Speed             208
Current Description    208
dtype: int64

In [16]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [17]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Atuona,PF,79.75,light rain,-9.8000,-139.0333,
5,Vaini,TO,73.40,broken clouds,-21.2000,-175.2000,
9,Rikitea,PF,76.33,light rain,-23.1203,-134.9692,
14,Lastoursville,GA,73.40,fog,-0.8143,12.7287,
15,Butaritari,KI,82.35,moderate rain,3.0707,172.7902,
16,Hilo,US,77.00,overcast clouds,19.7297,-155.0900,
23,Busselton,AU,77.00,few clouds,-33.6500,115.3333,
31,Kingsville,US,78.80,overcast clouds,27.5159,-97.8561,
33,Poum,NC,76.15,scattered clouds,-20.2333,164.0167,
34,Matamoros,MX,79.00,overcast clouds,25.5333,-103.2500,


In [18]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found.... skipping")        

Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping


In [19]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Atuona,PF,79.75,light rain,-9.8000,-139.0333,Villa Enata
5,Vaini,TO,73.40,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
9,Rikitea,PF,76.33,light rain,-23.1203,-134.9692,Pension Maro'i
14,Lastoursville,GA,73.40,fog,-0.8143,12.7287,Auberge Bon Coin 02
15,Butaritari,KI,82.35,moderate rain,3.0707,172.7902,Isles Sunset Lodge


In [20]:
# 7a. Before exporting double check Dataframe is correct.
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Atuona,PF,79.75,light rain,-9.8000,-139.0333,Villa Enata
5,Vaini,TO,73.40,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
9,Rikitea,PF,76.33,light rain,-23.1203,-134.9692,Pension Maro'i
14,Lastoursville,GA,73.40,fog,-0.8143,12.7287,Auberge Bon Coin 02
15,Butaritari,KI,82.35,moderate rain,3.0707,172.7902,Isles Sunset Lodge
16,Hilo,US,77.00,overcast clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
23,Busselton,AU,77.00,few clouds,-33.6500,115.3333,Observatory Guest House
31,Kingsville,US,78.80,overcast clouds,27.5159,-97.8561,Econo Lodge Kingsville
33,Poum,NC,76.15,scattered clouds,-20.2333,164.0167,Gîte kajeon
34,Matamoros,MX,79.00,overcast clouds,25.5333,-103.2500,Quinta Belem


In [21]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [22]:
# 9. Using the template add hotel name, city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [23]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))